# Instalar Bibliotecas

In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


# Importar Bibliotecas

In [3]:
import sidrapy
import requests, json
import pyarrow as pa
import pyarrow.parquet as pq
import ibge.localidades as api_ibge

# VARIAVEIS GLOBAIS

In [4]:
PATH = "C:\\PROJETOS\\ibge_integration\\RAW_DATA\\" # SUBSTITUA UM PATH PARA UM DE SUA PREFERENCIA OU UM BUCKET

# Funções

In [5]:
# Função que converte o json em parquet
def json_to_parquet(json_data, parquet_filename):
    try:
        json_formated = {chave: [d[chave] for d in json_data] for chave in json_data[0].keys()}
       
        table = pa.Table.from_pydict(json_formated)
        
        with pq.ParquetWriter(parquet_filename, schema=table.schema) as writer:
            writer.write_table(table)

        print(f"Arquivo Parquet '{parquet_filename}' gerado com sucesso.")

        # Validando se os dados foram salvos corretamente lendo o arquivo Parquet
        tabela_lida = pq.read_table(parquet_filename)
        
        return True

    except Exception as e:
        print(f"Erro ao salvar ou ler o arquivo Parquet: {e}")
        return False

# Estados e Regiões

In [6]:
# Lendo dados de regioes pelo ibge e jogando no lake (RAW_DATA)
json_regioes = api_ibge.Regioes().json()
json_to_parquet(json_data=json_regioes,parquet_filename=f'{PATH}\\regioes.parquet')

# Lendo dados de estado pelo ibge e jogando no lake (RAW_DATA)
json_estados = api_ibge.Estados().json()
json_to_parquet(json_data=json_estados,parquet_filename=f'{PATH}\\estados.parquet')


Arquivo Parquet 'C:\PROJETOS\ibge_integration\RAW_DATA\\regioes.parquet' gerado com sucesso.
Arquivo Parquet 'C:\PROJETOS\ibge_integration\RAW_DATA\\estados.parquet' gerado com sucesso.


True

# IDH

In [14]:
url_idh = "https://servicodados.ibge.gov.br/api/v1/pesquisas/37/resultados/{estado}"

for estado in json_estados:
    result = requests.get(url_idh.format(estado = estado['id']))
    if result.status_code == 200:
        print(result.content)

b'[{"id":30255,"res":[{"localidade":"11","res":{"1991":"0.407","2000":"0.537","2010":"0.690","2012":"0.694","2013":"0.687","2014":"0.692","2015":"0.706","2016":"0.715","2017":"0.721","2018":"0.73","2019":"0.73","2020":"0.739","2021":"0.7"},"notas":{"1991":null,"2000":null,"2010":null,"2012":null,"2013":null,"2014":null,"2015":null,"2016":null,"2017":null,"2018":null,"2019":null,"2020":null,"2021":null}}]}]'
b'[{"id":30255,"res":[{"localidade":"12","res":{"1991":"0.402","2000":"0.517","2010":"0.663","2012":"0.701","2013":"0.706","2014":"0.715","2015":"0.718","2016":"0.723","2017":"0.712","2018":"0.733","2019":"0.739","2020":"0.746","2021":"0.71"},"notas":{"1991":null,"2000":null,"2010":null,"2012":null,"2013":null,"2014":null,"2015":null,"2016":null,"2017":null,"2018":null,"2019":null,"2020":null,"2021":null}}]}]'
b'[{"id":30255,"res":[{"localidade":"13","res":{"1991":"0.430","2000":"0.515","2010":"0.674","2012":"0.691","2013":"0.702","2014":"0.706","2015":"0.711","2016":"0.711","2017":

# POPULACIONAL

# PIB